In [50]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json
import time

In [51]:
cid = "f9f71ea70ee04fd8a216eab5ad4d9eee"
secret = "86266ce20917448694cb69e96b50c2c6"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
# sp.new_releases(country='US', limit=5, offset=0)

In [52]:
producer = KafkaProducer(bootstrap_servers=['0.0.0.0'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [53]:
# Function to retrieve new releases in the US
def fetch_new_releases(limit, offset):
    new_releases = sp.new_releases(country='US', limit=limit, offset=offset)
    album_uris = [album['uri'] for album in new_releases['albums']['items']]
    return album_uris

# Function to retrieve tracks for an album
def retrieve_tracks(album_uri):
    tracks_response = sp.album_tracks(album_uri)
    tracks = tracks_response['items']
    album_name = sp.album(album_uri)['name']  # Fetch album name using album URI
    for track in tracks:
        track['album_name'] = album_name  # Add album name to each track
    return tracks

# Function to retrieve audio features for a track
def audio_features(track_uri):
    features = sp.audio_features(track_uri)
    if features:
        track_info = sp.track(track_uri)
        popularity = track_info['popularity']
        if popularity:
            features[0]['popularity'] = popularity  # Add popularity to the features dictionary
        return features[0]
    else:
        return None

# Main function to fetch data for new releases

def fetch_and_ingest_new_release_data(limit, existing_tracks):
    new_data = []
    unique_tracks_fetched = 0  # Initialize counter for unique tracks fetched
    offset = 0
    while unique_tracks_fetched < 2000:
        album_uris = fetch_new_releases(limit, offset)
        if not album_uris:  # No more new releases
            print("No new releases found.")
            break
        for album_uri in album_uris:
            tracks = retrieve_tracks(album_uri)
            print(tracks)
            for track in tracks:
                track_uri = track['uri']
                if track_uri not in existing_tracks:
                    track_features = audio_features(track_uri)
                    if track_features:
                        track_id = track_features['id']
                        track_name = track['name']
                        album_name = track['album_name'] if 'album_name' in track else None
                        artist_name = track['artists'][0]['name']
                        genres = sp.artist(track['artists'][0]['id'])['genres']
                        data = {
                            'Track ID': track_id,
                            'Track Name': track_name,
                            'Album': album_name,
                            'Artist': artist_name,
                            'Genres': genres,
                            'Track URI': track_uri,
                            'Audio Features': track_features
                        }
                        new_data.append(data)
                        existing_tracks.add(track_uri)
                        unique_tracks_fetched += 1  # Increment the counter
                        print(data)
                        producer.send('spotify_topic', value=data)
                        producer.flush()

        offset += limit 
    return new_data
      
existing_tracks = set()    
fetch_and_ingest_new_release_data(5, existing_tracks)

[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1RyvyyTE3xzB2ZywiAwp0i'}, 'href': 'https://api.spotify.com/v1/artists/1RyvyyTE3xzB2ZywiAwp0i', 'id': '1RyvyyTE3xzB2ZywiAwp0i', 'name': 'Future', 'type': 'artist', 'uri': 'spotify:artist:1RyvyyTE3xzB2ZywiAwp0i'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/0iEtIxbK0KxaSlF7G42ZOp'}, 'href': 'https://api.spotify.com/v1/artists/0iEtIxbK0KxaSlF7G42ZOp', 'id': '0iEtIxbK0KxaSlF7G42ZOp', 'name': 'Metro Boomin', 'type': 'artist', 'uri': 'spotify:artist:0iEtIxbK0KxaSlF7G42ZOp'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK',

[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6M2wZ9GZgrQXHCFfjv46we'}, 'href': 'https://api.spotify.com/v1/artists/6M2wZ9GZgrQXHCFfjv46we', 'id': '6M2wZ9GZgrQXHCFfjv46we', 'name': 'Dua Lipa', 'type': 'artist', 'uri': 'spotify:artist:6M2wZ9GZgrQXHCFfjv46we'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BZ', 'CA', 'CD', 'CG', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', 'MN', 'MO', 'MR', 'MT', 'MU', 'MV', 'MW', 'MX', 'M

{'Track ID': '59QO0lJJgJdHt36rfBdKoG', 'Track Name': 'Illusion - Extended', 'Album': 'Illusion', 'Artist': 'Dua Lipa', 'Genres': ['dance pop', 'pop', 'uk pop'], 'Track URI': 'spotify:track:59QO0lJJgJdHt36rfBdKoG', 'Audio Features': {'danceability': 0.692, 'energy': 0.767, 'key': 11, 'loudness': -5.786, 'mode': 1, 'speechiness': 0.0407, 'acousticness': 0.0765, 'instrumentalness': 0.000159, 'liveness': 0.0985, 'valence': 0.416, 'tempo': 126.964, 'type': 'audio_features', 'id': '59QO0lJJgJdHt36rfBdKoG', 'uri': 'spotify:track:59QO0lJJgJdHt36rfBdKoG', 'track_href': 'https://api.spotify.com/v1/tracks/59QO0lJJgJdHt36rfBdKoG', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/59QO0lJJgJdHt36rfBdKoG', 'duration_ms': 263840, 'time_signature': 4}}
{'Track ID': '6qszJ4PtfQSe9Fs3A0zIIi', 'Track Name': 'Illusion - Instrumental', 'Album': 'Illusion', 'Artist': 'Dua Lipa', 'Genres': ['dance pop', 'pop', 'uk pop'], 'Track URI': 'spotify:track:6qszJ4PtfQSe9Fs3A0zIIi', 'Audio Features': {'dancea

{'Track ID': '0pkiYzXyU58nKV74xsAuwW', 'Track Name': 'MERCEDES (feat. Oscar Maydon)', 'Album': 'MERCEDES (feat. Oscar Maydon)', 'Artist': 'Becky G', 'Genres': ['latin pop', 'latin viral pop', 'rap latina', 'reggaeton', 'trap latino', 'urbano latino'], 'Track URI': 'spotify:track:0pkiYzXyU58nKV74xsAuwW', 'Audio Features': {'danceability': 0.762, 'energy': 0.815, 'key': 9, 'loudness': -6.176, 'mode': 0, 'speechiness': 0.0331, 'acousticness': 0.383, 'instrumentalness': 3.85e-06, 'liveness': 0.216, 'valence': 0.934, 'tempo': 125.918, 'type': 'audio_features', 'id': '0pkiYzXyU58nKV74xsAuwW', 'uri': 'spotify:track:0pkiYzXyU58nKV74xsAuwW', 'track_href': 'https://api.spotify.com/v1/tracks/0pkiYzXyU58nKV74xsAuwW', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0pkiYzXyU58nKV74xsAuwW', 'duration_ms': 168360, 'time_signature': 3, 'popularity': 31}}
[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2HPaUgqeutzr3jx5a9WyDV'}, 'href': 'https://api.spotify.com/v1

{'Track ID': '7nnSTOoBWl2ctLxoqgCZoq', 'Track Name': 'L o s e M y M i n d', 'Album': 'L o s e M y M i n d', 'Artist': 'PARTYNEXTDOOR', 'Genres': ['r&b', 'rap', 'urban contemporary'], 'Track URI': 'spotify:track:7nnSTOoBWl2ctLxoqgCZoq', 'Audio Features': {'danceability': 0.433, 'energy': 0.477, 'key': 4, 'loudness': -8.028, 'mode': 0, 'speechiness': 0.0448, 'acousticness': 0.0583, 'instrumentalness': 1.85e-05, 'liveness': 0.228, 'valence': 0.144, 'tempo': 170.312, 'type': 'audio_features', 'id': '7nnSTOoBWl2ctLxoqgCZoq', 'uri': 'spotify:track:7nnSTOoBWl2ctLxoqgCZoq', 'track_href': 'https://api.spotify.com/v1/tracks/7nnSTOoBWl2ctLxoqgCZoq', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7nnSTOoBWl2ctLxoqgCZoq', 'duration_ms': 183991, 'time_signature': 4, 'popularity': 38}}
{'Track ID': '7vRNfBcZAT969BKodO9Iiv', 'Track Name': 'R e a l W o m a n', 'Album': 'L o s e M y M i n d', 'Artist': 'PARTYNEXTDOOR', 'Genres': ['r&b', 'rap', 'urban contemporary'], 'Track URI': 'spotify:tra

...

{'Track ID': '2jTuy90JMdLuDtSQyjLaiQ', 'Track Name': 'My Home', 'Album': 'You Promised A Lifetime', 'Artist': 'Myles Smith', 'Genres': ['singer-songwriter pop'], 'Track URI': 'spotify:track:2jTuy90JMdLuDtSQyjLaiQ', 'Audio Features': {'danceability': 0.753, 'energy': 0.485, 'key': 11, 'loudness': -9.431, 'mode': 1, 'speechiness': 0.035, 'acousticness': 0.345, 'instrumentalness': 0.00104, 'liveness': 0.192, 'valence': 0.547, 'tempo': 95.968, 'type': 'audio_features', 'id': '2jTuy90JMdLuDtSQyjLaiQ', 'uri': 'spotify:track:2jTuy90JMdLuDtSQyjLaiQ', 'track_href': 'https://api.spotify.com/v1/tracks/2jTuy90JMdLuDtSQyjLaiQ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2jTuy90JMdLuDtSQyjLaiQ', 'duration_ms': 206250, 'time_signature': 4, 'popularity': 51}}
{'Track ID': '3nsVmer6MSwhmau3I1nRzI', 'Track Name': 'River', 'Album': 'You Promised A Lifetime', 'Artist': 'Myles Smith', 'Genres': ['singer-songwriter pop'], 'Track URI': 'spotify:track:3nsVmer6MSwhmau3I1nRzI', 'Audio Features':

{'Track ID': '4pAJsiM3G2RyJ8A2mrOT20', 'Track Name': 'Intro to Everything', 'Album': 'All I Ever Want Is Everything', 'Artist': 'Blu DeTiger', 'Genres': [], 'Track URI': 'spotify:track:4pAJsiM3G2RyJ8A2mrOT20', 'Audio Features': {'danceability': 0.325, 'energy': 0.31, 'key': 7, 'loudness': -12.159, 'mode': 1, 'speechiness': 0.0309, 'acousticness': 0.0603, 'instrumentalness': 0.821, 'liveness': 0.121, 'valence': 0.203, 'tempo': 128.685, 'type': 'audio_features', 'id': '4pAJsiM3G2RyJ8A2mrOT20', 'uri': 'spotify:track:4pAJsiM3G2RyJ8A2mrOT20', 'track_href': 'https://api.spotify.com/v1/tracks/4pAJsiM3G2RyJ8A2mrOT20', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4pAJsiM3G2RyJ8A2mrOT20', 'duration_ms': 32901, 'time_signature': 5, 'popularity': 34}}
{'Track ID': '5T2RWhA9lQCy3rPyQnjePB', 'Track Name': 'Latency', 'Album': 'All I Ever Want Is Everything', 'Artist': 'Blu DeTiger', 'Genres': [], 'Track URI': 'spotify:track:5T2RWhA9lQCy3rPyQnjePB', 'Audio Features': {'danceability': 0.6

{'Track ID': '0ho6tGnCPHu7qKPBmqJwM3', 'Track Name': 'Sad Girl Machine', 'Album': 'All I Ever Want Is Everything', 'Artist': 'Blu DeTiger', 'Genres': [], 'Track URI': 'spotify:track:0ho6tGnCPHu7qKPBmqJwM3', 'Audio Features': {'danceability': 0.887, 'energy': 0.545, 'key': 10, 'loudness': -4.348, 'mode': 0, 'speechiness': 0.163, 'acousticness': 0.448, 'instrumentalness': 9.93e-05, 'liveness': 0.173, 'valence': 0.684, 'tempo': 142.074, 'type': 'audio_features', 'id': '0ho6tGnCPHu7qKPBmqJwM3', 'uri': 'spotify:track:0ho6tGnCPHu7qKPBmqJwM3', 'track_href': 'https://api.spotify.com/v1/tracks/0ho6tGnCPHu7qKPBmqJwM3', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ho6tGnCPHu7qKPBmqJwM3', 'duration_ms': 208524, 'time_signature': 4, 'popularity': 31}}
{'Track ID': '3YqS0F3Mf9LP9Yi7WpLxTF', 'Track Name': 'Overdrive', 'Album': 'All I Ever Want Is Everything', 'Artist': 'Blu DeTiger', 'Genres': [], 'Track URI': 'spotify:track:3YqS0F3Mf9LP9Yi7WpLxTF', 'Audio Features': {'danceability': 0

[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7bcbShaqKdcyjnmv4Ix8j6'}, 'href': 'https://api.spotify.com/v1/artists/7bcbShaqKdcyjnmv4Ix8j6', 'id': '7bcbShaqKdcyjnmv4Ix8j6', 'name': 'St. Vincent', 'type': 'artist', 'uri': 'spotify:artist:7bcbShaqKdcyjnmv4Ix8j6'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD',

[{'Track ID': '59xD5osEFsaNt5PXfIKUnX',
  'Track Name': 'Illusion',
  'Album': 'Illusion',
  'Artist': 'Dua Lipa',
  'Genres': ['dance pop', 'pop', 'uk pop'],
  'Track URI': 'spotify:track:59xD5osEFsaNt5PXfIKUnX',
  'Audio Features': {'danceability': 0.679,
   'energy': 0.772,
   'key': 8,
   'loudness': -4.721,
   'mode': 0,
   'speechiness': 0.0472,
   'acousticness': 0.0277,
   'instrumentalness': 2.59e-06,
   'liveness': 0.106,
   'valence': 0.709,
   'tempo': 126.964,
   'type': 'audio_features',
   'id': '59xD5osEFsaNt5PXfIKUnX',
   'uri': 'spotify:track:59xD5osEFsaNt5PXfIKUnX',
   'track_href': 'https://api.spotify.com/v1/tracks/59xD5osEFsaNt5PXfIKUnX',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/59xD5osEFsaNt5PXfIKUnX',
   'duration_ms': 188143,
   'time_signature': 4}},
 {'Track ID': '59QO0lJJgJdHt36rfBdKoG',
  'Track Name': 'Illusion - Extended',
  'Album': 'Illusion',
  'Artist': 'Dua Lipa',
  'Genres': ['dance pop', 'pop', 'uk pop'],
  'Track URI': 'spotif

In [ ]:
global_dataframe

In [67]:
dataframe.columns

Index(['Track ID', 'Track Name', 'Artist', 'Genres', 'Track URI',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'popularity'],
      dtype='object')

In [9]:
dataframe["Album"].value_counts()

Series([], Name: Album, dtype: int64)

In [ ]:
# def continuously_fetch_and_ingest(limit):
#     existing_tracks = set()
#     while True:
#         new_release_data = fetch_and_ingest_new_release_data(limit, existing_tracks)
# #         print(new_release_data)
#         for data in new_release_data:
#             producer.send('spotify_topic', value=data)
#         producer.flush()

In [18]:
df = pd.DataFrame({'col_A':[1,5,7,8],'col_B':[9,7,4,3]})
df.to_csv("inputt.csv")